# Vienna RNA Package Testing

This notebook provides simple implementations of the RNA secondary structure prediction capabilities offered by the ViennaRNA Package Version 2.

Many of these examples are taken directly from the package's documentation:
- API reference: https://www.tbi.univie.ac.at/RNA/ViennaRNA/doc/html/index.html
- Script Interface Documentation: https://www.tbi.univie.ac.at/RNA/ViennaRNA/doc/html/wrappers.html
- Python Examples: https://www.tbi.univie.ac.at/RNA/ViennaRNA/doc/html/examples_python.html

## Setup

### Import Required Packages

In [1]:
# built-in packages
import sys

# third-party packages
import Bio.SeqIO

# the Vienna RNA python package is automatically installed in a particular directory
# ensure that that directory is on Python's path
VIENNA_DIR = '/usr/local/lib/python3.7/site-packages'
sys.path.append(VIENNA_DIR)
import RNA

### Load Example Sequence

In [2]:
# load the PGI gene's FASTA file to use as an example sequence
pgi_seq_record = Bio.SeqIO.read('pgi.fasta', 'fasta')
example_seq = str(pgi_seq_record.seq)[:25]

## Basic Calculations

Also shows off different plotting capabilities

### Minimum Free Energy Structure

In [3]:
# compute minimum free energy (MFE) and corresponding structure
fc = RNA.fold_compound(example_seq)
# compute minimum free energy (mfe) and corresponding structure
(ss, mfe) = fc.mfe()
RNA.svg_rna_plot(example_seq, ss, 'mfe.svg')
print(f'{example_seq}\n{ss} [ {mfe:.2f} kcal/mol]')

GGATCTGGCGAAAGAGTGCGATCTG
(((((..((......))..))))). [ -6.80 kcal/mol]


<img src="mfe.svg">

In [4]:
with open('loop_energies.txt', 'w', encoding='utf-8') as fwrite:
    ss_mfe = fc.eval_structure_verbose(ss, fwrite)


with open('loop_energies.txt', 'r') as fread:
    for line in fread:
        print(line)

External loop                           :   -20

Interior loop (  1, 24) GT; (  2, 23) GC:  -210

Interior loop (  2, 23) GC; (  3, 22) AT:  -240

Interior loop (  3, 22) AT; (  4, 21) TA:  -110

Interior loop (  4, 21) TA; (  5, 20) CG:  -240

Interior loop (  5, 20) CG; (  8, 17) GT:    80

Interior loop (  8, 17) GT; (  9, 16) CG:  -250

Hairpin  loop (  9, 16) CG              :   310



### MFE for Multiple Sequences

In [5]:
sequences = [
    "CUGCCUCACAACGUUUGUGCCUCAGUUACCCGUAGAUGUAGUGAGGGU",
    "CUGCCUCACAACAUUUGUGCCUCAGUUACUCAUAGAUGUAGUGAGGGU",
    "---CUCGACACCACU---GCCUCGGUUACCCAUCGGUGCAGUGCGGGU"
]
# compute the consensus sequence
cons = RNA.consensus(sequences)

# predict Minmum Free Energy and corresponding secondary structure
(ss, mfe) = RNA.alifold(sequences)

RNA.file_PS_aln('mfe_aln.ps', sequences, ['s1', 's2', 's3'], ss, 0)

# print output
print("%s\n%s [ %6.2f ]" % (cons, ss, mfe))

CUGCCUCACAACAUUUGUGCCUCAGUUACCCAUAGAUGUAGUGAGGGU
...((((((.(((((((((...........))))))))).)))))).. [ -10.86 ]


<img src="mfe_aln.png">

### Changing Defaults

In [6]:
# The RNA sequence
seq = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
# create a new model details structure
md = RNA.md()
# change temperature and dangle model
md.temperature = 20.0 # 20 Deg Celcius
md.dangles     = 1    # Dangle Model 1
# create a fold compound
fc = RNA.fold_compound(seq, md)
# predict Minmum Free Energy and corresponding secondary structure
(ss, mfe) = fc.mfe()
# print sequence, structure and MFE
print("%s\n%s [ %6.2f ]\n" % (seq, ss, mfe))

GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA
..(((((........)))))(((((((...)))))))..... [ -13.43 ]



### Suboptimal Structure Prediction

In [7]:
# Set global switch for unique ML decomposition
RNA.cvar.uniq_ML = 1

subopt_data = {'counter': 1, 'sequence': example_seq}

# Print a subopt result as FASTA record
def print_subopt_result(structure, energy, data):
    if not structure == None:
        print(">subopt %d" % data['counter'])
        print("%s" % data['sequence'])
        print("%s [%6.2f]" % (structure, energy))
        # increase structure counter
        data['counter'] = data['counter'] + 1

# Create a 'fold_compound' for our sequence
fold_comp = RNA.fold_compound(example_seq)

# Enumerate all structures 500 dacal/mol = 5 kcal/mol arround
# the MFE and print each structure using the function above
fold_comp.subopt_cb(500, print_subopt_result, subopt_data)

>subopt 1
GGATCTGGCGAAAGAGTGCGATCTG
((.((..((......))..)).)). [ -2.10]
>subopt 2
GGATCTGGCGAAAGAGTGCGATCTG
(((..(.(((......))).)))). [ -2.00]
>subopt 3
GGATCTGGCGAAAGAGTGCGATCTG
(((.(..((......))..).))). [ -2.10]
>subopt 4
GGATCTGGCGAAAGAGTGCGATCTG
((((...((......))...)))). [ -2.80]
>subopt 5
GGATCTGGCGAAAGAGTGCGATCTG
((((...((........)).)))). [ -2.20]
>subopt 6
GGATCTGGCGAAAGAGTGCGATCTG
((((...(((......))).)))). [ -2.50]
>subopt 7
GGATCTGGCGAAAGAGTGCGATCTG
((((.((((......)).)))))). [ -1.80]
>subopt 8
GGATCTGGCGAAAGAGTGCGATCTG
(((((..............))))). [ -2.70]
>subopt 9
GGATCTGGCGAAAGAGTGCGATCTG
(((((...(....).....))))). [ -2.20]
>subopt 10
GGATCTGGCGAAAGAGTGCGATCTG
(((((...(......)...))))). [ -3.10]
>subopt 11
GGATCTGGCGAAAGAGTGCGATCTG
(((((...(........).))))). [ -1.90]
>subopt 12
GGATCTGGCGAAAGAGTGCGATCTG
(((((...((......)).))))). [ -2.20]
>subopt 13
GGATCTGGCGAAAGAGTGCGATCTG
(((((..(........)..))))). [ -2.00]
>subopt 14
GGATCTGGCGAAAGAGTGCGATCTG
(((((..((......))..))))). [ -6.80]
>

### RNAfold -p –MEA equivalent 

In [8]:
seq = "AUUUCCACUAGAGAAGGUCUAGAGUGUUUGUCGUUUGUCAGAAGUCCCUAUUCCAGGUACGAACACGGUGGAUAUGUUCGACGACAGGAUCGGCGCACUA"
# create fold_compound data structure (required for all subsequently applied  algorithms)
fc = RNA.fold_compound(seq)
# compute MFE and MFE structure
(mfe_struct, mfe) = fc.mfe()
# rescale Boltzmann factors for partition function computation
fc.exp_params_rescale(mfe)
# compute partition function
(pp, pf) = fc.pf()
# compute centroid structure
(centroid_struct, dist) = fc.centroid()
# compute free energy of centroid structure
centroid_en = fc.eval_structure(centroid_struct)
# compute MEA structure
(MEA_struct, MEA) = fc.MEA()
# compute free energy of MEA structure
MEA_en = fc.eval_structure(MEA_struct)
# print everything like RNAfold -p --MEA
print("%s\n%s (%6.2f)" % (seq, mfe_struct, mfe))
print("%s [%6.2f]" % (pp, pf))
print("%s {%6.2f d=%.2f}" % (centroid_struct, centroid_en, dist))
print("%s {%6.2f MEA=%.2f}" % (MEA_struct, MEA_en, MEA))
print(" frequency of mfe structure in ensemble %g; ensemble diversity %-6.2f" % (fc.pr_structure(mfe_struct), fc.mean_bp_distance()))

AUUUCCACUAGAGAAGGUCUAGAGUGUUUGUCGUUUGUCAGAAGUCCCUAUUCCAGGUACGAACACGGUGGAUAUGUUCGACGACAGGAUCGGCGCACUA
.......(((((.....)))))(((((..((((((((((.......(((.....)))..((((((.........))))))..))))))..))))))))). (-22.20)
.......(((((.....)))))(((((..(((({(((((.,..{,,{{{..,{.|,|,.{{{{{{,...,},,.})})))}})))))}..))))))))). [-24.66]
.......(((((.....)))))(((((..((((.(((((.....................((((...........))))...)))))...))))))))). {-15.20 d=14.12}
.......(((((.....)))))(((((..((((.(((((.......(.........)..((((((.........))))))..)))))...))))))))). {-16.60 MEA=75.49}
 frequency of mfe structure in ensemble 0.0183287; ensemble diversity 22.18 
